In [9]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# ✅ Load YOLOv8 model (you can change to yolov8s.pt for better accuracy)
model = YOLO("yolov8n.pt")

# ✅ Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)

# ✅ Load your video
video_path = "/home/rguktrkvalley/Desktop/projects/AI/broadcast.mp4"
cap = cv2.VideoCapture(video_path)

# ✅ Optional: Save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_broadcast.mp4", fourcc, 20.0,
                      (int(cap.get(3)), int(cap.get(4))))

# ✅ Frame reading loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ Detect objects with YOLOv8
    results = model(frame)[0]

    detections = []
    for result in results.boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        conf = float(result.conf[0])
        cls = int(result.cls[0])
        # ✅ Only track "person" class
        if model.names[cls] == 'person':
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # ✅ Track using DeepSORT
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        # ✅ Draw bounding box + Player ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'Player_{track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # ✅ Show live output
    cv2.imshow("Player Re-ID", frame)

    # ✅ Save output
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ✅ Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 1 sports ball, 261.8ms
Speed: 4.9ms preprocess, 261.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 sports ball, 406.7ms
Speed: 10.6ms preprocess, 406.7ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 sports ball, 213.9ms
Speed: 10.1ms preprocess, 213.9ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 sports ball, 1 tennis racket, 207.3ms
Speed: 5.3ms preprocess, 207.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 sports ball, 329.0ms
Speed: 7.6ms preprocess, 329.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 222.3ms
Speed: 5.3ms preprocess, 222.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 195.7ms
Speed: 4.7ms preprocess, 195.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

